In [ ]:
REQUEST_ID = 0
AOI = 'POLYGON ((-94.74437712304278 42.10688312211514, -94.78351591698579 42.10713783113902, -94.78351591698579 42.13795006926095, -94.74437712304278 42.13756819109344, -94.74437712304278 42.10688312211514))'
START_DATE = '2020-07-07'
END_DATE = '2020-07-14'

In [ ]:
import os
import shutil
import time
import numpy as np
import geopandas as gpd
import rasterio.mask
from tqdm.notebook import tqdm
import shapely.wkt
from pathlib import Path

from code.download.utils import get_tiles, check_nodata
from code.download.load_tiles import load_images
from code.index_research import calculate_ndvi
from code.plant_stress import PlantStress
from code.utils import dump_no_data_geosjon, stitch_tiles

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [ ]:
SEED = 66
NB_USER = os.getenv('NB_USER')
BASE = os.getcwd()
DATA_DIR = f"/home/{NB_USER}/work"
API_KEY = os.path.join(DATA_DIR, ".secret/sentinel2_google_api_key.json")
LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
RESULTS_DIR = os.path.join(DATA_DIR, f"results/dd/plant_stress_multi_field/{REQUEST_ID}")
SENTINEL_TILES_PATH = os.path.join(BASE, "sentinel2grid.geojson")
PB_PATH = os.path.join(BASE, "iowa_fields.geojson")
NAME = 'Field anomalies'
PB_PATH



BANDS = {'B04','B08'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

for pbd_file in os.listdir(RESULTS_DIR):
    if pbd_file.startswith(str(REQUEST_ID)+'_') or pbd_file.startswith('demo_'+str(REQUEST_ID)+'_'):
        PB_PATH = os.path.join(RESULTS_DIR, pbd_file)
        break
        
    else:
        # should be trigger PBDNN notebook excecution? PBDNN pipeline?
        PB_PATH = aoi_filename
        
PB_PATH
os.makedirs(RESULTS_DIR, exist_ok=True)

In [ ]:
b04_tiles, b08_tiles, tci_tiles = [], [], []
date_tile_info = get_tiles(aoi_filename, SENTINEL_TILES_PATH)

In [ ]:
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)
needed_num_tiles = len(checked.keys())

while len(b04_tiles) < needed_num_tiles and len(b08_tiles) < needed_num_tiles:
    if CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] > 70:
        metadata = dict(START_DATE=START_DATE, END_DATE=END_DATE, REQUEST_ID=REQUEST_ID, NAME=NAME)
        geojson_path = os.path.join(RESULTS_DIR, f"{START_DATE}_{END_DATE}_no_data.geojson")
        dump_no_data_geosjon(polygon, geojson_path, metadata)
        raise ValueError("Images not loaded for given AOI. Change dates, constraints")

    for i, tile in date_tile_info.iterrows():
        if len(checked[tile.tileID]) < 1:
            continue
                
        tile_folder = Path(sorted(checked[tile.tileID])[0])
        tile_files = list(tile_folder.rglob('*.jp2'))
        b04_tiles.append([x for x in tile_files if '_B04_' in str(x)][0])
        b08_tiles.append([x for x in tile_files if '_B08_' in str(x)][0])
        print(f'filtered: {tile_folder}')
        
    CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] += 5
    CONSTRAINTS['CLOUDY_PIXEL_PERCENTAGE'] += 5
    loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
    checked = check_nodata(loadings, PRODUCT_TYPE)

In [ ]:
if len(b04_tiles) > 1:
    b04_tile = stitch_tiles(b04_tiles, str(b04_tiles[0]).replace('.jp2', '_merged.tif'))
    b08_tile = stitch_tiles(b08_tiles, str(b08_tiles[0]).replace('.jp2', '_merged.tif'))
else:
    b04_tile = str(b04_tiles[0])
    b08_tile = str(b08_tiles[0])

In [ ]:
b04_tile

In [ ]:
os.makedirs(f'{RESULTS_DIR}/plant_stress/{REQUEST_ID}', exist_ok=True)
ndvi_path = f'{RESULTS_DIR}/plant_stress/{REQUEST_ID}/{REQUEST_ID}_ndvi.tif'
if not os.path.exists(ndvi_path):
    calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

In [ ]:
fields = gpd.read_file(PB_PATH)

In [ ]:
with rasterio.open(ndvi_path) as src:
    target_crs = src.crs

In [ ]:
fields = gpd.read_file(PB_PATH)

In [ ]:
min_ndvi = 0.3
z_score = 5
z_score_anom = 1
rasters_to_stitch = []
p_bar = tqdm(fields.iterrows(), total=len(fields))
colors = {"Normal Growth": (0, 0, 0), "Anomaly": (182, 10, 28)}
ps = PlantStress(min_ndvi=min_ndvi, noise_z_score=z_score, anomaly_z_score=z_score_anom)

for i, record in enumerate(p_bar):
    idx, row = record
    field = fields.to_crs(target_crs).iloc[idx].geometry
    out_raster = ps.segment_field(field, ndvi_path, START_DATE, END_DATE, REQUEST_ID, idx)
    rasters_to_stitch.append(out_raster)

In [ ]:
rasters_to_stitch

In [ ]:
path = f'{RESULTS_DIR}/plant_stress/{REQUEST_ID}/{REQUEST_ID}_merged.tif'
stitch_tiles(rasters_to_stitch, path)

In [ ]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        shutil.rmtree(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')
        
    try:
        os.remove(ndvi_path)
    except FileNotFoundError:
        print(f'Error removing file {ndvi_path}')
        
    for i in range(len(b04_tiles)):
        ndvi_p = f'{RESULTS_DIR}/plant_stress/{REQUEST_ID}/ndvi_{i}.tif'
        try:
            os.remove(ndvi_p)
        except FileNotFoundError:
            print(f'Error removing file {ndvi_p}')
            
    for raster_path in rasters_to_stitch:
        try:
            os.remove(raster_path)
        except FileNotFoundError:
            print(f'Error removing file {raster_path}')